In [1]:
import os

from series_extraction.excel_loader import ExcelLoader
from series_extraction.excel_validator import ExcelValidator
from series_extraction.excel_cleaner import ExcelCleaner
from series_extraction.table_finder import TableFinder
from series_extraction.series_extractor import SeriesExtractor
from series_extraction.excel_compatibility_checker import ExcelCompatibilityChecker
from series_extraction.series_iterator import SeriesIterator
from series_extraction.series_mapper import SeriesMapper

from ast_building.formula_parser import FormulaParser
from ast_building.series_implementer import SeriesImplementer

from ast_transformation.formula_generator import FormulaGenerator

In [2]:
current_directory = os.getcwd()

parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
data_directory = os.path.join(parent_directory, 'data')

project_name = 'test_excel_3'

excel_raw_file_path = os.path.join(data_directory, "excel_files_raw", f"{project_name}_raw.xlsx")
excel_reduced_filepath = os.path.join(data_directory, "excel_files_reduced", f"{project_name}_reduced.xlsx")

In [3]:
excel_raw= ExcelLoader.load_file(excel_raw_file_path)
excel_reduced = ExcelLoader.load_file(excel_reduced_filepath)

In [4]:
is_valid = ExcelValidator.validate_excel(excel_reduced)

In [5]:
if not is_valid:
    raise Exception("Excel file is not valid")

In [6]:
excel_reduced_clean = ExcelCleaner.clean_excel(excel_reduced)

In [7]:
extracted_tables, data = TableFinder.find_tables(excel_reduced_clean)

In [8]:
series_data = SeriesExtractor.extract_table_details(extracted_tables, data)

In [9]:
is_compatible = ExcelCompatibilityChecker.check_file(excel_raw, excel_reduced, extracted_tables)

In [10]:
if not is_compatible:
    raise Exception("Excel file is not compatible")

In [11]:
series_dict = SeriesExtractor.extract_series(extracted_tables=extracted_tables, data=data)

In [12]:
series_dict

{'toyota_21_analysis': [Series(series_id='toyota_21_analysis|Created Month|1|1', worksheet=Worksheet(sheet_name='toyota_21_analysis', workbook_file_path=None, worksheet=None), series_header='Created Month', formulas=[None, None], values=[44531, 44501], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
  Series(series_id='toyota_21_analysis|Revenue after insurance & CDW|1|2', worksheet=Worksheet(sheet_name='toyota_21_analysis', workbook_file_path=None, worksheet=None), series_header='Revenue after insurance & CDW', formulas=[None, None], values=[18820.579999999998, 9567.82], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=2, coordinate='B2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.FLOAT: 'float'>),
  Series(series_id='toyota_21_analysis|Toyota Revenue|1|3', workshe

In [13]:
series_mapping = SeriesMapper.map_series(series_dict)

In [14]:
series_mapping

{Worksheet(sheet_name='toyota_21_analysis', workbook_file_path=None, worksheet=None): {Cell(column=1, row=2, coordinate=None, value=None, value_type=None): (0,
   Series(series_id='toyota_21_analysis|Created Month|1|1', worksheet=Worksheet(sheet_name='toyota_21_analysis', workbook_file_path=None, worksheet=None), series_header='Created Month', formulas=[None, None], values=[44531, 44501], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A2', value=None, value_type=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)),
  Cell(column=1, row=3, coordinate=None, value=None, value_type=None): (1,
   Series(series_id='toyota_21_analysis|Created Month|1|1', worksheet=Worksheet(sheet_name='toyota_21_analysis', workbook_file_path=None, worksheet=None), series_header='Created Month', formulas=[None, None], values=[44531, 44501], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=1, row=2, coordinate='A

In [15]:
series_iterator = SeriesIterator.iterate_series(series_dict)

In [16]:
series_list = [series for series in series_iterator if series.formulas != [None, None]]

Series Id can be concatenation of sheet_name, header_name, header_column_index, header_row_index

In [17]:
for series in series_list:
    formula_1 = series.formulas[0]
    formula_1_ast = FormulaParser.parse_formula(formula_1)
    series_implementer = SeriesImplementer(series_mapping, sheet_name = series.worksheet.sheet_name)
    formula_1_ast_series = series_implementer.replace_range_nodes(formula_1_ast)

    formula_2 = series.formulas[1]
    formula_2_ast = FormulaParser.parse_formula(formula_2)
    formula_2_ast_series = series_implementer.replace_range_nodes(formula_2_ast)


    print(formula_1)
    print(formula_2)

    ast_generator = FormulaGenerator.get_ast_generator(formula_1_ast_series, formula_2_ast_series)
    
    ast_delta = ast_generator.get_ast_with_deltas()
    ast_delta_serialised = SeriesImplementer.serialise_ast_to_formula(ast_delta)
    print(ast_delta_serialised)
    

=SUM(C2,E2,F2)
=SUM(C3,E3,F3)
node and length
(('toyota_21_analysis|Toyota Revenue|1|3',), (0, 0))
2
node and length
(('toyota_21_analysis|50% of Parking Costs|1|5',), (0, 0))
2
node and length
(('toyota_21_analysis|50% of Adhoc|1|6',), (0, 0))
2
SUM(('toyota_21_analysis|Toyota Revenue|1|3',), (1, 1, 0, 0), 0, ('toyota_21_analysis|50% of Parking Costs|1|5',), (1, 1, 0, 0), 0, ('toyota_21_analysis|50% of Adhoc|1|6',), (1, 1, 0, 0), 0)
=-F2+-E2
=-F3+-E3
node and length
(('toyota_21_analysis|50% of Adhoc|1|6',), (0, 0))
2
node and length
(('toyota_21_analysis|50% of Parking Costs|1|5',), (0, 0))
2
(( - ('toyota_21_analysis|50% of Adhoc|1|6',), (1, 1, 0, 0), 0) + ( - ('toyota_21_analysis|50% of Parking Costs|1|5',), (1, 1, 0, 0), 0))
=B2-D2-G2
=B3-D3-G3
node and length
(('toyota_21_analysis|Revenue after insurance & CDW|1|2',), (0, 0))
2
node and length
(('toyota_21_analysis|Payment to TGB minus costs|1|4',), (0, 0))
2
node and length
(('toyota_21_analysis|Total Costs (50%)|1|7',), (0, 0))

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'